In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
### Установим красивые дефолтные настройки
### Может быть лень постоянно прописывать
### У графиков параметры цвета, размера, шрифта
### Можно положить их в словарь дефолтных настроек

import matplotlib as mlp

mlp.rcParams['lines.linewidth'] = 5

mlp.rcParams['xtick.major.size'] = 20
mlp.rcParams['xtick.major.width'] = 5
mlp.rcParams['xtick.labelsize'] = 20
mlp.rcParams['xtick.color'] = '#FF5533'

mlp.rcParams['ytick.major.size'] = 20
mlp.rcParams['ytick.major.width'] = 5
mlp.rcParams['ytick.labelsize'] = 20
mlp.rcParams['ytick.color'] = '#FF5533'

mlp.rcParams['axes.labelsize'] = 20
mlp.rcParams['axes.titlesize'] = 20
mlp.rcParams['axes.titlecolor'] = '#00B050'
mlp.rcParams['axes.labelcolor'] = '#00B050'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### RecSys

In [4]:
### Загружаем датасет
### Информация по фильмам: movieid-название-жанр

movies = pd.read_csv('movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
### Информация по оценкам: userid-movied-рейтинг-дата

ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### Коллаборативная фильтрация

In [6]:
### Разделим выборку на train-test

ratings = ratings.sort_values("timestamp")

train = ratings.iloc[:-20000].copy()
test = ratings.iloc[-20000:].copy()

In [7]:
train.sort_values("timestamp")

,userId,movieId,rating,timestamp
66719,429,595,5.0,828124615
66714,429,468,3.0,828124615
66662,429,22,4.0,828124615
66665,429,150,5.0,828124615
66667,429,161,5.0,828124615
...,...,...,...,...
79613,495,6157,2.0,1458636454
79575,495,3825,2.0,1458636458
79569,495,3157,0.5,1458636467
79616,495,6383,3.0,1458636473


In [8]:
test.sort_values("timestamp")

,userId,movieId,rating,timestamp
79534,495,1405,0.5,1458636491
79535,495,1438,3.5,1458636501
79576,495,3826,3.5,1458636502
79506,495,160,3.0,1458636521
79537,495,1499,2.5,1458636525
...,...,...,...,...
81475,514,187031,2.5,1537674927
81477,514,187595,3.0,1537674946
81336,514,5247,2.5,1537757040
81335,514,5246,1.5,1537757059


In [9]:
### Реализуем user-based подход

pivot = train.pivot_table(index='movieId', 
                          columns='userId',
                          values='rating')

corrs = pivot.corr()

corrs

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29,31,32,33,34,35,36,37,38,39,40,42,43,44,45,46,48,51,53,54,55,56,57,58,59,60,61,63,64,66,68,69,70,71,72,74,75,76,78,79,80,81,82,83,84,85,86,87,88,90,91,92,93,94,95,96,97,99,100,101,102,103,104,105,107,108,109,110,112,113,115,116,117,118,119,120,121,123,124,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,142,143,144,145,146,147,149,150,151,152,155,156,157,158,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,185,186,187,188,189,191,192,193,194,195,197,198,199,200,201,202,203,204,206,207,208,211,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,249,250,253,254,255,256,257,259,260,261,262,263,264,265,266,267,268,269,270,271,273,274,275,276,277,278,280,281,282,283,284,285,286,287,288,289,...,315,316,317,318,320,321,322,323,324,325,326,327,330,332,333,334,335,336,337,340,341,342,343,344,345,346,347,348,349,350,351,353,354,355,356,357,358,359,360,361,364,366,367,368,369,370,371,372,373,374,375,376,377,378,379,381,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,402,403,404,405,406,407,409,410,411,412,414,415,416,419,420,421,422,423,424,425,426,427,428,429,430,431,432,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,460,462,463,464,465,466,467,468,469,470,472,473,474,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,516,518,520,521,522,524,527,528,529,530,531,532,533,535,536,537,538,539,540,541,542,543,544,545,546,547,552,553,554,555,557,558,559,562,563,565,566,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,584,585,587,588,589,590,591,592,593,594,595,597,598,600,601,602,603,604,605,606,607,608,609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,NaN,0.079819,0.207983,0.268749,-0.291636,-0.118773,0.469668,0.918559,-0.037987,-0.087643,NaN,0.878310,0.229081,NaN,0.191851,3.790087e-02,0.261338,0.325180,0.419641,0.074475,-0.463039,-0.364372,0.141421,0.189437,-0.014257,-7.049479e-02,0.033366,0.277017,0.088523,0.032356,0.297044,0.432426,-0.419355,0.246085,-0.225681,-3.540052e-01,0.140809,0.111291,0.684448,0.267402,8.333333e-02,-0.102544,0.065183,NaN,-0.058222,5.136540e-01,0.260000,0.354260,0.351998,7.534646e-02,NaN,1.563150e-01,0.284589,0.201428,0.155762,0.028222,0.301511,-0.020870,-0.730487,0.596432,-0.061679,0.454726,0.362147,0.279620,-0.088798,0.229552,-0.238751,0.103293,0.149626,-0.025999,NaN,0.237428,NaN,0.217643,8.215838e-01,0.095662,-1.000000,0.194307,-0.303642,0.235706,0.087576,-0.372104,0.583450,0.148877,0.200675,-0.312718,0.026359,0.079559,0.004742,0.230089,0.051848,-0.053379,1.126229e-01,0.374241,0.540833,0.265191,0.309517,0.145793,-2.483351e-16,0.151194,0.594874,-0.273326,2.500000e-01,0.371999,0.391287,NaN,-0.247207,-1.162755e-01,9.124486e-02,0.048696,0.061674,0.454677,-0.255745,0.307293,0.382456,0.095141,0.390567,0.790569,0.044025,1.797385e-17,-0.169526,0.207225,0.366900,1.000000,-0.394874,-0.095924,-0.418330,0.128698,-0.186321,0.183611,0.006667,0.801784,-0.970725,0.322824,-0.816497,0.708333,0.422577,-0.239501,0.361752,0.172031,0.405284,0.347585,0.251706,0.288136,0.367895,0.487688,0.617213,-0.173384,NaN,0.539360,0.113508,0.561341,-0.117454,NaN,-0.103535,0.181824,-0.245253,4.483219e-01,-0.013524,0.213508,-0.250000,0.353553,0.178647,-2.490677e-02,0.162120,-0.072548,-0.225514,

In [10]:
corrs = (
    corrs
    .stack()
    .rename_axis(['userId1', 'userId2'])
    .reset_index()
)

corrs.columns = ['userId1', 'userId2', 'corr']

In [11]:
corrs = corrs[corrs['corr'] >= 0]

corrs

,userId1,userId2,corr
0,1,1,1.000000
1,1,3,0.079819
2,1,4,0.207983
3,1,5,0.268749
6,1,8,0.469668
...,...,...,...
192255,609,604,0.641624
192257,609,606,0.533002
192258,609,607,0.190117
192259,609,608,0.488929


In [12]:
### Для каждого юзера из теста 
### Найдем всех "соседей"
### Которые смотрели те же фильмы,
### Что и юзер на тесте


import math

preds = []

for user in test['userId'].unique():
    
    ### Если юзера не было в трейне,
    ### То прогноз в выбранной парадигме дать не сможем
    
    if user in train['userId'].unique():
        part = test[test['userId']==user]

        ### Выделим соседей данного юзера
        
        neighbours = corrs[corrs['userId1']==user]
        neighbours_users = neighbours['userId2'].unique()
        
        ### Если соседей нет, то и предсказывать нечего
        ### Разве что среднее выбранного юзера по фильмам
        ### Но это сильно тупо
        
        if neighbours_users.shape[0]==0:
            continue
        
        ### Выделим фильмы, для которых нужно дать прогноз
        
        movies_ = part['movieId'].unique()

        ### Выделим часть данных с трейна про соседей
        
        train_part = train[train['userId'].isin(neighbours_users)]
        
        ### Посчитаем средние оценки соседей
        
        neighbours_means = train_part.groupby('userId')['rating'].mean()
        
        ### Присоединим эту информацию и посчитаем
        ### Остальные компоненты формулы для рассчета предсказания
        ### Относительно соседей и фильмов,
        ### Для которых возможно сделать прогноз
        
        train_part = train_part[train_part['movieId'].isin(movies_)]
        train_part = pd.merge(train_part,
                              neighbours[['userId2', 'corr']],
                              right_on='userId2',
                              left_on='userId',
                              how='left')
        
        train_part['neighbour_mean'] = train_part['userId2'].map(neighbours_means)
        train_part['diff'] = train_part['rating'] - train_part['neighbour_mean']
        train_part['diff_dot_corr'] = train_part['diff'] * train_part['corr']
        
        ### Посчитаем среднее по юзеру
    
        user_mean = train[train['userId']==user]['rating'].mean()
        
        ### Применим формулу для предсказания 
        
        upper_part = train_part.groupby('movieId')['diff_dot_corr'].sum()
        lower_part = train_part.groupby('movieId')['corr'].sum()
        
        predictions = upper_part / lower_part + user_mean
        predictions = predictions.reset_index()
        predictions.columns = ['movieId', 'prediction']
        predictions['userId'] = user
        
        preds.append(predictions)
        
preds = pd.concat(preds)

preds = pd.merge(
                    preds,
                    test[['userId', 'movieId', 'rating']],
                    on=['userId', 'movieId'],
                    how='left'
)

In [13]:
print(f"""Смогли дать предсказания только для {preds.shape[0]} 
          пар айтем-юзер из {test.shape[0]} тестовых""")

preds

Смогли дать предсказания только для 1125 
          пар айтем-юзер из 20000 тестовых


,movieId,prediction,userId,rating
0,160,3.254725,495,3.0
1,1059,4.379525,495,5.0
2,1172,4.714483,495,5.0
3,1405,3.663485,495,0.5
4,1438,3.517763,495,3.5
...,...,...,...,...
1120,122886,4.360065,68,4.5
1121,122904,4.098191,68,4.0
1122,134130,3.431193,68,5.0
1123,139385,4.208013,68,3.0


In [14]:
### Посчитаем DSG@2 хотя бы для этих пар!

users_dsgs = []

for user in preds['userId'].unique():
    part = preds[preds['userId']==user]
    part = part.sort_values('prediction', ascending=False)
    part = part.reset_index()
    user_dsg2 = (np.log2(part.index+1) * part.rating)[:2].sum()
    
    users_dsgs.append(user_dsg2)

print(f"Среднее DSG@2 по пользователям из теста: {np.mean(users_dsgs)}")

Среднее DSG@2 по пользователям из теста: 3.9444444444444446


### Контентный подход

In [15]:
df = pd.merge(
    ratings,
    movies,
    on='movieId',
    how='left'
)

df.head()

,userId,movieId,rating,timestamp,title,genres
0,429,595,5.0,828124615,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX
1,429,588,5.0,828124615,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
2,429,590,5.0,828124615,Dances with Wolves (1990),Adventure|Drama|Western
3,429,592,5.0,828124615,Batman (1989),Action|Crime|Thriller
4,429,432,3.0,828124615,City Slickers II: The Legend of Curly's Gold (...,Adventure|Comedy|Western


In [16]:
### У всех фильмов есть жанры и названия

df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

In [17]:
### Выделим год из названия фильмов

import re

def find_num(st):
    
    nums_list = re.findall(r'\d+', st)
    
    if len(nums_list) > 0:
        return nums_list[-1]
    else:
        return '0'

def filter_missing_data(num):
    if num > 1900:
        return num
    else:
        return 2000
    
df['movieYear'] = df['title'].apply(lambda x: filter_missing_data(int(find_num(x))))

df.head()

,userId,movieId,rating,timestamp,title,genres,movieYear
0,429,595,5.0,828124615,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,1991
1,429,588,5.0,828124615,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,1992
2,429,590,5.0,828124615,Dances with Wolves (1990),Adventure|Drama|Western,1990
3,429,592,5.0,828124615,Batman (1989),Action|Crime|Thriller,1989
4,429,432,3.0,828124615,City Slickers II: The Legend of Curly's Gold (...,Adventure|Comedy|Western,1994


In [18]:
### Сделаем One-Hot-Encoding жанров!

all_genres = ['Adventure', 'Comedy', 'Action', 'Mystery', 'Crime', 'Thriller',
              'Drama', 'Animation', 'Children', 'Horror', 'Documentary',
              'Sci-Fi', 'Fantasy', 'Film-Noir', 'Western', 'Musical', 'Romance',
              '(no genres listed)', 'War']

for genre in all_genres:
    df[genre] = (
        df['genres']
        .str
        .contains(genre)
        .apply(int)
    )
    
df = df.drop('genres', axis=1)

df.head()

,userId,movieId,rating,timestamp,title,movieYear,Adventure,Comedy,Action,Mystery,Crime,Thriller,Drama,Animation,Children,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,(no genres listed),War
0,429,595,5.0,828124615,Beauty and the Beast (1991),1991,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0
1,429,588,5.0,828124615,Aladdin (1992),1992,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0
2,429,590,5.0,828124615,Dances with Wolves (1990),1990,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
3,429,592,5.0,828124615,Batman (1989),1989,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,429,432,3.0,828124615,City Slickers II: The Legend of Curly's Gold (...,1994,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [19]:
### Разделим на трейн-тест

train_new = df.iloc[:-20000].copy()
test_new = df.iloc[-20000:].copy()

In [20]:
### Пока что имеем в качестве описания контента
### Только какую-то информацию о фильмах

train_new

,userId,movieId,rating,timestamp,title,movieYear,Adventure,Comedy,Action,Mystery,Crime,Thriller,Drama,Animation,Children,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,(no genres listed),War
0,429,595,5.0,828124615,Beauty and the Beast (1991),1991,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0
1,429,588,5.0,828124615,Aladdin (1992),1992,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0
2,429,590,5.0,828124615,Dances with Wolves (1990),1990,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
3,429,592,5.0,828124615,Batman (1989),1989,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,429,432,3.0,828124615,City Slickers II: The Legend of Curly's Gold (...,1994,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80831,495,6157,2.0,1458636454,Daredevil (2003),2003,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
80832,495,3825,2.0,1458636458,Coyote Ugly (2000),2000,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
80833,495,3157,0.5,1458636467,Stuart Little (1999),1999,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
80834,495,6383,3.0,1458636473,"2 Fast 2 Furious (Fast and the Furious 2, The)...",2003,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
### Ну хоть что-то о юзерах хочется добавить!
### Например, сколько фильмов они посмотрели
### За тренировочный период
### И какие средние оценки получились (с шумом!)

user_count_views = train_new.groupby('userId').size()
user_means = train_new.groupby('userId')['rating'].mean()

train_new['userViews'] = train_new['userId'].map(user_count_views)

noise = np.random.normal(0, 0.1, [train_new.shape[0],])
train_new['userMeans'] = train_new['userId'].map(user_means) + noise

In [22]:
### Посчитаем среднее кол-во просмотров всех юзеров
### И среднюю оценку по средним оценкам всех юзеров

overall_views_mean = int(user_count_views.mean())
overall_meanrating_mean = int(user_means.mean())

test_new['userViews'] = (
    test_new['userId']
    .map(user_count_views)
    .fillna(overall_views_mean)
)


test_new['userMeans'] = (
    test_new['userId']
    .map(user_means)
    .fillna(overall_meanrating_mean)
)

train_new = train_new.drop(['userId', 'movieId',
                            'timestamp', 'title'], axis=1)

test_new = test_new.drop(['userId', 'movieId',
                          'timestamp', 'title'], axis=1)

In [23]:
train_new

,rating,movieYear,Adventure,Comedy,Action,Mystery,Crime,Thriller,Drama,Animation,Children,Horror,Documentary,Sci-Fi,Fantasy,Film-Noir,Western,Musical,Romance,(no genres listed),War,userViews,userMeans
0,5.0,1991,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0,58,3.825581
1,5.0,1992,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,58,4.005040
2,5.0,1990,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,58,3.991826
3,5.0,1989,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,58,3.926699
4,3.0,1994,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,58,4.023305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80831,2.0,2003,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,197,3.917296
80832,2.0,2000,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,197,4.085134
80833,0.5,1999,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,197,3.906800
80834,3.0,2003,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,197,4.039212


In [24]:
X_train = train_new.drop('rating', axis=1)
X_test = test_new.drop('rating', axis=1)

y_train = train_new['rating']
y_test = test_new['rating']

In [25]:
from catboost import CatBoostRegressor, Pool

catboost = CatBoostRegressor()


catboost.fit(X_train,
             y_train,
             cat_features=['movieYear'],
             )

Learning rate set to 0.081956
0:	learn: 1.0204553	total: 73.7ms	remaining: 1m 13s
1:	learn: 1.0092183	total: 84.2ms	remaining: 42s
2:	learn: 0.9994140	total: 95.7ms	remaining: 31.8s
3:	learn: 0.9910091	total: 107ms	remaining: 26.7s
4:	learn: 0.9837945	total: 119ms	remaining: 23.7s
5:	learn: 0.9775266	total: 129ms	remaining: 21.4s
6:	learn: 0.9716394	total: 144ms	remaining: 20.4s
7:	learn: 0.9668690	total: 154ms	remaining: 19s
8:	learn: 0.9623729	total: 163ms	remaining: 18s
9:	learn: 0.9584834	total: 174ms	remaining: 17.2s
10:	learn: 0.9547956	total: 184ms	remaining: 16.5s
11:	learn: 0.9518524	total: 194ms	remaining: 15.9s
12:	learn: 0.9489754	total: 204ms	remaining: 15.5s
13:	learn: 0.9465777	total: 214ms	remaining: 15s
14:	learn: 0.9446518	total: 224ms	remaining: 14.7s
15:	learn: 0.9427141	total: 233ms	remaining: 14.3s
16:	learn: 0.9409174	total: 242ms	remaining: 14s
17:	learn: 0.9391302	total: 253ms	remaining: 13.8s
18:	learn: 0.9375360	total: 262ms	remaining: 13.5s
19:	learn: 0.9363

In [26]:
test_new = df.iloc[-20000:].copy()

X_test['pred'] = catboost.predict(X_test)
X_test['target'] = y_test
X_test['userId'] = test_new['userId']
X_test['movieId'] = test_new['movieId']

In [27]:
users_dsgs = []

for user in X_test['userId'].unique():
    part = X_test[X_test['userId']==user]
    part = part.sort_values('pred')
    part = part.reset_index()
    user_dsg2 = (np.log2(part.index+1) * part.target)[:2].sum()
    
    users_dsgs.append(user_dsg2)
    
print(f"Среднее DSG@2 по пользователям из теста: {np.mean(users_dsgs)}")

Среднее DSG@2 по пользователям из теста: 3.3275862068965516


In [28]:
preds

,movieId,prediction,userId,rating
0,160,3.254725,495,3.0
1,1059,4.379525,495,5.0
2,1172,4.714483,495,5.0
3,1405,3.663485,495,0.5
4,1438,3.517763,495,3.5
...,...,...,...,...
1120,122886,4.360065,68,4.5
1121,122904,4.098191,68,4.0
1122,134130,3.431193,68,5.0
1123,139385,4.208013,68,3.0


In [29]:
new_preds = pd.merge(preds,
                     X_test[['userId', 'movieId', 'pred']],
                     on=['userId', 'movieId'],
                     how='left')

users_dsgs = []

for user in new_preds['userId'].unique():
    part = new_preds[new_preds['userId']==user]
    part = part.sort_values('pred')
    part = part.reset_index()
    user_dsg2 = (np.log2(part.index+1) * part.rating)[:2].sum()

    users_dsgs.append(user_dsg2)
    
print(f"Среднее DSG@2 по пользователям из теста, которые были в трейне: {np.mean(users_dsgs)}")

Среднее DSG@2 по пользователям из теста, которые были в трейне: 3.0925925925925926
